In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

df = pd.read_csv(r"C:\Users\san\Documents\Data\Numbers.csv" , encoding = "ISO-8859-1")
df.head(5)

,Department,Comments,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,0,We hosted Top Academic Achievers of Mwakirunge...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Good 0 and motivational leadership is key...wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,It is with deep sorrow that we announce the pa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Our 3rd Year statistics students toured Kenya ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Visit by the Kenya Space Weather team to launc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info

<bound method DataFrame.info of      Department                                           Comments Unnamed: 2  \
0             0  We hosted Top Academic Achievers of Mwakirunge...        NaN   
1             0  Good 0 and motivational leadership is key...wi...        NaN   
2             0  It is with deep sorrow that we announce the pa...        NaN   
3             1  Our 3rd Year statistics students toured Kenya ...        NaN   
4             0  Visit by the Kenya Space Weather team to launc...        NaN   
..          ...                                                ...        ...   
184           5  We will help you explain the short roles on yo...        NaN   
185           1  To the math and computer science courses. Some...        NaN   
186           3  Remember to always carry your school ID with y...        NaN   
187           1  To sum it all the advice remained that student...        NaN   
188           1  Exams irregularities is one the pandemic that ...        NaN

In [4]:
df['Department'] = df['Department'].astype(int)
df['Department'].value_counts()

0    45
1    42
3    40
5    27
6    17
2    12
4     6
Name: Department, dtype: int64

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

token = tokenizer.encode_plus(
    df['Comments'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)



token.input_ids
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Comments'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [6]:
labels = np.zeros((len(df), 7))
labels.shape

(189, 7)

In [7]:
labels[np.arange(len(df)), df['Department'].values] = 1
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(7,), dtype=tf.float64, name=None))>

In [9]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

dataset = dataset.map(SentimentDatasetMapFunction)
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(7,), dtype=tf.float64, name=None))>

In [10]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)

In [11]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 7), dtype=tf.float64, name=None))>

In [12]:
p = 0.8
train_size = int((len(df)//16)*p)

train_size

8

In [13]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)
from transformers import TFBertModel

model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [14]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(7, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [16]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=8
)


sentiment_model.save('TTu_model1')

Epoch 1/8
8/8 [==============================] - 1439s 181s/step - loss: 1.3064 - accuracy: 0.5938 - val_loss: 1.2154 - val_accuracy: 0.7500
Epoch 2/8
8/8 [==============================] - 860s 102s/step - loss: 1.1797 - accuracy: 0.7109 - val_loss: 1.0732 - val_accuracy: 0.7083
Epoch 3/8
8/8 [==============================] - 762s 98s/step - loss: 1.0294 - accuracy: 0.7500 - val_loss: 0.9447 - val_accuracy: 0.7708
Epoch 4/8
8/8 [==============================] - 1211s 163s/step - loss: 0.9804 - accuracy: 0.7109 - val_loss: 0.8489 - val_accuracy: 0.7292
Epoch 5/8
8/8 [==============================] - 683s 84s/step - loss: 0.8912 - accuracy: 0.7578 - val_loss: 0.6744 - val_accuracy: 0.8542
Epoch 6/8
8/8 [==============================] - 1012s 133s/step - loss: 0.7737 - accuracy: 0.8047 - val_loss: 0.5973 - val_accuracy: 0.8542
Epoch 7/8
8/8 [==============================] - 1079s 138s/step - loss: 0.6669 - accuracy: 0.8281 - val_loss: 0.5570 - val_accuracy: 0.8750
Epoch 8/8
8/8 [===

INFO:tensorflow:Assets written to: TTu_model1\assets


INFO:tensorflow:Assets written to: TTu_model1\assets


In [19]:
sentiment_model = tf.keras.models.load_model('TTu_model1')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Administration', 'Academics', 'Elections', 'Health', 'Extra-curricular' , 'Spam', 'Sports']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]


input_text = input('Enter comment here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

Enter comment here:  Greetings from Taita Taveta University; The University Bus will be available at Voi town to feryy First year Students and Parents free of charge on Monday (20/09/2021) from 6am. For those who will be arriving with the SGR at 12noon the University Bus will be picking you up at the SGR Voi station. Welcome to Taita Taveta University; Home of Ideas


Predicted Sentiment: Administration
